<a href="https://colab.research.google.com/github/oroering8/Bio108Tutorial/blob/main/Week10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#How much overlap exists between critical habitat and protected areas in the US? How good of a job are we doing at protecting endangered species?
#Do protected areas have higher levels of vegetation and endangered species? Are our limited conservaiton resources going to right places/working?
#I think I'm leaning towards solely pursuing the first question, it seems like it might be too complicated to try for both.
#Method: Compare a national park to a nearby undeveloped and unprotected area. How much overlap exists between critical habitat and protected areas.
#Reasoning: Higher levels of endagnered species shows that this is in fact improved habitat for the at risk species.
#Analyses: comparision of raster density bewteen protected and nearby unprotected areas. Graph of critical habitat and protected area overlap. Percentage overlap in US.

In [ ]:
#importing packages
! pip install rasterio
! pip install rasterstats

In [ ]:
#importing packages
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import geopandas as gpd
import rasterio
import rasterio.plot
import rasterio.mask
import rasterio.features
import rasterstats
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
from rasterio.crs import CRS

In [ ]:
#setting up google drive with my files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#loading in the boundary files using gpd.read_file, and then reading in the raster data that has information about species richness.
#I'm aiming to understand whether critcal habitat and protected areas have high levels of overlap, and am using yosemite as an example to start.
#got my data on species richness from IUCN, and the yosemite shapefile from the previous HW. I then will create a plot of the same area nearby
#on the the leafmap and analyse species richness
yosemite = gpd.read_file('/content/drive/MyDrive/yosemite_boundary/yose_boundary.shp')
species_richness = rasterio.open('/content/drive/MyDrive/Species_Richness/Combined_SR_2024.tif')
protected_areas = gpd.read_file('https://www.usgs.gov/programs/gap-analysis-project/science/protected-areas')
critical_habitat = gpd.read_file('/content/drive/MyDrive/crithab_all_layers.zip')

In [ ]:
#seeing which CRS my files are in
print(species_richness.crs)
print(yosemite.crs)
print(critical_habitat.crs)
print(protected_areas.crs)

In [1]:
#installing necessary leafmap packages
! pip install -U leafmap
! pip install fiona
! pip install folium matplotlib mapclassify
! pip install localtileserver
! pip install rasterio

import leafmap
import fiona
import geopandas as gpd

#zooming in on the desired area
m = leafmap.Map(center=[40, -100], zoom=4)
url = "https/endangered_species_richness"
#adding a baselayer to the map
m.add_wms_layer(
    url=url,
    layers="USGSNAIPImagery:FalseColorComposite",
    name="NAIP",
    attribution="USGS",
    format="image/png",
    shown=True,
)
#adding the USGS layer to visualize species richness
m.add_wms_layer(
    url=url,
    layers="Imagery",
    name="Endagered Species",
    attribution="MRLC",
    format="image/png",
    shown=True,
)
m.add_legend(title="NLCD Land Cover Type", builtin_legend="NLCD")
#creating a vector to roughly define an area outside the park
m.add_markers(markers=[[40, -100], [45, -110], [50, -120]])
#creating a vector to roughly define the bounds of yosemite, as I'm having trouble loading in the exact shapefile
m.add_markers(markers=[[40, -100], [45, -110], [50, -120]])
#displaying the plot
m


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.8/523.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [ ]:
#calculating overlap
intersection = critical_habitat.intersection(protected_areas)
overlap_area = intersection.area
print("Overlap area:", overlap_area)
print("Protected area:", protected_areas.area)
print("Critical_habitat area:", critical_habitat.area)
#calculating the proportion of critical habitat that is also a protected area
proportion = critical_habitat.area/(critical_habitat.area + protected_areas.area)

In [ ]:
# Define the target CRS
target_crs = species_richness.crs

# Reproject the yosemite NP in the new CRS
yosemite_reproj = yosemite.to_crs(target_crs)

In [ ]:
#checking the bounds so I know the range I can clip
species_richness.shape

In [ ]:
#ran into trouble with this code. Could not get it to work, but ideally I would play around with bounds to clip the raster data to be just
#the area around yosemite. I would be then overlaying the shapefile of the bounds, so that I could visualize in detail and not just see the whole globe
subset = species_richness[:, 200:300, 0:1200].clip(0, 1)
rgb_subset = np.dstack((subset[4], subset[3], subset[2]))
rgb_subset.shape

In [ ]:
#The plot works, but I couldn't get it to zoom so it just shows the whole world even though I just need the small area. I'm assuming the vector
#for the park bounds is in the but just too small to see currently
fig, ax = plt.subplots()
rasterio.plot.show(species_richness, ax=ax)
yosemite.plot(ax=ax, color='black', edgecolor='white')

In [ ]:
#Analysis for the overlap between critical habitat and protected areas.
# Create two sets of polygons
polygons_set1 = gpd.GeoDataFrame({
    'id': [1, 2],
    'geometry': [
        (critical_habitat),
    ]
})

polygons_set2 = gpd.GeoDataFrame({
    'id': [1, 2],
    'geometry': [
        (protected_areas)
    ]
})

# Create a map
m = leafmap.Map(center=(1.5, 1.5), zoom=10)

# Add polygons to map
m.add_gdf(polygons_set1, layer_name="Critical Habitat", style={"color": "blue", "fillOpacity": 0.3})
m.add_gdf(polygons_set2, layer_name="Protected Areas", style={"color": "red", "fillOpacity": 0.3})

# Display map
m

In [ ]:
# Calculating intersections between all pairs. Got some help from stack exchange on this one.
overlap_areas = []
for poly1 in polygons_set1.geometry:
    for poly2 in polygons_set2.geometry:
        intersection = poly1.intersection(poly2)
        if not intersection.is_empty:
            overlap_areas.append(intersection.area)

total_overlap_area = sum(overlap_areas)

print(f"Total Overlap Area: {total_overlap_area}")